In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
input_shape = (3, 256, 256)
output_shape = 4

def cnn_final_convolution_size(initial_size, kernel_size, pooling_size):
    size = initial_size
    size = size - kernel_size + 1
    size = size // pooling_size
    size = size - kernel_size + 1
    size = size // pooling_size
    return size

Basic Model

In [5]:
class MRI_CNN(nn.Module):
    def __init__(self, kernel_size=5, conv_out_features_1=32, conv_out_features_2=64, pooling_size=2, linear_size_1=512, linear_size_2=128):
        super(MRI_CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[0], conv_out_features_1, kernel_size)
        self.pool = nn.MaxPool2d(pooling_size, pooling_size)
        self.conv2 = nn.Conv2d(conv_out_features_1, conv_out_features_2, kernel_size)
        self.fc1 = nn.Linear(
            conv_out_features_2 * cnn_final_convolution_size(input_shape[1], kernel_size, pooling_size) * cnn_final_convolution_size(input_shape[2], kernel_size, pooling_size),
            linear_size_1) 
        self.fc2 = nn.Linear(linear_size_1, linear_size_2)
        self.fc3 = nn.Linear(linear_size_2, output_shape) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x